### RIOT API RATE LIMITS:
- 20 requests every 1 seconds(s) <br>
- 100 requests every 2 minutes(s) -> 1 request/1.2s

In [1]:
import requests
import pandas as pd
import os
import time
from bs4 import BeautifulSoup

In [ ]:
API_KEY = '' # klucz do api, dostepny na stronie developer.riotgames.com
ROOT_URL = f'https://europe.api.riotgames.com'

In [ ]:
# sciagniecie puuid gracza na podstawie jego nazwy i tag line
def get_player_puuid(game_name, tag_line):
    global ROOT_URL, API_KEY

    endpoint = f'/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={API_KEY}'
    response = requests.get(ROOT_URL + endpoint)
    return response.json()['puuid']

In [ ]:
# sciagniecie historii gier wybranego gracza na podstawie jego numeru puuid
def get_match_history_ids(puuid, count=100):
    global ROOT_URL, API_KEY

    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    params = f'?count={count}&api_key={API_KEY}'
    response = requests.get(ROOT_URL + endpoint + params)
    return response.json()

In [ ]:
# sciagnicie jsona z danymi z gry na podstawie id meczu
def get_match_data_from_id(match_id):
    global ROOT_URL, API_KEY

    endpoint = f'/lol/match/v5/matches/{match_id}?api_key={API_KEY}'
    response = requests.get(ROOT_URL + endpoint)
    return response.json()

In [ ]:
# procesowanie danych sciagnietych z gry dla jednego gracza i zaladowanie ich do dataframe
def process_match_json_for_one_player(match_json, puuid):
    side_dict = {
        100:'blue',
        200:'red'
    }

    try:
        info = match_json['info']

        metadata = match_json['metadata']
        matchId = metadata['matchId']
        participants = metadata['participants']

        player = info['participants'][participants.index(puuid)]

        gameStartTimestamp = info['gameStartTimestamp']
        gameEndTimestamp = info['gameEndTimestamp']
        gameDuration = info['gameDuration']
        timePlayed = (gameEndTimestamp-gameStartTimestamp)/1000
        gameMode = info['gameMode']
        queueId = info['queueId']
        platformId = info['platformId']
    
        puuid = player['puuid']
        riotIdGameName = player['summonerName']
        try:
            riotIdTagLine = player['riotIdTagline']
        except:
            riotIdTagLine = ''
        side = side_dict[player['teamId']]
        win = player['win']

        champion = player['championName']
        kills = player['kills']
        deaths = player['deaths']
        assists = player['assists']
        earlySurrender = player['gameEndedInEarlySurrender']
        surrender = player['gameEndedInSurrender']
        dragonKills = player['dragonKills']
        baronKills = player['baronKills']
        champLevel = player['champLevel']
        inhibitorKills = player['inhibitorKills']
        inhibitorsLost = player['inhibitorsLost']
        itemsPurchased = player['itemsPurchased']
        damageDealtToBuildings = player['damageDealtToBuildings']
        damageDealtToObjectives = player['damageDealtToObjectives']
        goldEarned = player['goldEarned']
        lane = player['lane']
        objectivesStolen = player['objectivesStolen']
        totalMinionsKilled = player['totalMinionsKilled']
        totalAllyJungleMinionsKilled = player['totalAllyJungleMinionsKilled']
        totalEnemyJungleMinionsKilled = player['totalEnemyJungleMinionsKilled']
        totalNeutralMinionsKilled = totalAllyJungleMinionsKilled + totalEnemyJungleMinionsKilled
        totalDamageDealtToChampions = player['totalDamageDealtToChampions']
        totalDamageTaken = player['totalDamageTaken']
        totalTimeSpentDead = player['totalTimeSpentDead']
        turretKills = player['turretKills']
        turretsLost = player['turretsLost']
        visionScore = player['visionScore']
        wardsPlaced = player['wardsPlaced']

        matchDF = pd.DataFrame({
            'match_id': [matchId], 
            'participants': [participants], 
            'game_duration': [gameDuration],
            'game_duration_in_min': [round(gameDuration / 60, 2)],
            'game_mode': [gameMode],
            'queue_id': [queueId],
            'platform_id': [platformId],
            'puuid': [puuid],
            'riot_id': [riotIdGameName],
            'riot_tag': [riotIdTagLine],
            'time_played': [timePlayed],
            'side': [side],
            'win': [win],
            'early_surrender': [earlySurrender],
            'surrender': [surrender],
            'lane': [lane],
            'champion': [champion],
            'kills': [kills],
            'deaths': [deaths],
            'assists': [assists],
            'gold_earned': [goldEarned],
            'total_minions_killed': [totalMinionsKilled],
            'total_neutral_minions_killed': [totalNeutralMinionsKilled],
            'total_ally_jungle_minions_killed': [totalAllyJungleMinionsKilled],
            'total_enemy_jungle_minions_killed': [totalEnemyJungleMinionsKilled],
            'damage_dealt_to_buildings': [damageDealtToBuildings],
            'turret_kills': [turretKills],
            'turrets_lost': [turretsLost],
            'damage_dealt_to_objectives': [damageDealtToObjectives],
            'dragon_kills': [dragonKills],
            'objectives_stolen': [objectivesStolen],
            'total_damage_dealt_champions': [totalDamageDealtToChampions],
            'total_damage_taken': [totalDamageTaken],
            'total_time_spent_dead': [totalTimeSpentDead],
            'vision_score': [visionScore],
            'wards_placed': [wardsPlaced],
            'baron_kills': [baronKills],
            'champ_level': [champLevel],
            'inhibitor_kills': [inhibitorKills],
            'inhibitor_lost': [inhibitorsLost],
            'items_purchased': [itemsPurchased]
        })
    
        return matchDF
    except:
        return pd.DataFrame()

In [ ]:
# sprawdzenie czy wybrana gra jest gra rankingowa
def check_if_ranked(match_id):
    match_json = get_match_data_from_id(match_id=match_id)
    info = match_json['info']
    gameMode = info['gameMode']
    queueId = info['queueId']
    return (gameMode == 'CLASSIC' and queueId == 420) 

In [ ]:
# sciagniecie wszystkich graczy z meczu na podstawie id meczu
def get_match_participants_puuids(match_id):
    match_json = get_match_data_from_id(match_id=match_id)
    metadata = match_json['metadata']
    participants = metadata['participants']
    return participants

In [ ]:
# procesowanie danych dla calego meczu na podstawie danych poszczegolnych graczy
def process_match_json(match_id):
    global API_KEY, ROOT_URL

    match_json = get_match_data_from_id(match_id=match_id)
    participants_puuids = match_json['metadata']['participants']
    
    info = match_json['info']
    game_duration_in_ms = info['gameDuration']
    game_duration_in_min = round(game_duration_in_ms / 60, 2)
    game_mode = info['gameMode']
    queue_id = info['queueId']
    platform_id = info['platformId']

    win = ''
    surrender = False

    total_blue_kills = 0
    total_red_kills = 0
    total_blue_assists = 0
    total_red_assists = 0
    total_blue_gold_earned = 0
    total_red_gold_earned = 0
    total_blue_minions_killed = 0
    total_red_minions_killed = 0
    total_blue_turret_kills = 0
    total_red_turret_kills = 0
    total_blue_dragon_kills = 0
    total_red_dragon_kills = 0
    total_blue_damage_dealt_to_champions = 0
    total_red_damage_dealt_to_champions = 0
    total_blue_baron_kills = 0
    total_red_baron_kills = 0
    total_blue_champ_level = 0
    total_red_champ_level = 0
    total_blue_inhibitor_kills = 0
    total_red_inhibitor_kills = 0
    total_blue_items_purchased = 0
    total_red_items_purchased = 0 

    for participant_puuid in participants_puuids:
        player_df = process_match_json_for_one_player(match_json=match_json, puuid=participant_puuid)
        player_team = player_df['side'][0]
        
        if (player_df['win'][0] == True):
            win = player_df['side'][0]
        if (player_df['surrender'][0] == True):
            surrender = True

        if player_team == 'blue':
            total_blue_kills += player_df['kills'][0]
            total_blue_assists += player_df['assists'][0]
            total_blue_gold_earned += player_df['gold_earned'][0]
            total_blue_minions_killed += player_df['total_minions_killed'][0]
            total_blue_turret_kills += player_df['turret_kills'][0]
            total_blue_dragon_kills += player_df['dragon_kills'][0]
            total_blue_damage_dealt_to_champions += player_df['total_damage_dealt_champions'][0]
            total_blue_baron_kills += player_df['baron_kills'][0]
            total_blue_champ_level += player_df['champ_level'][0]
            total_blue_inhibitor_kills += player_df['inhibitor_kills'][0]
            total_blue_items_purchased += player_df['items_purchased'][0]
        else:
            total_red_kills += player_df['kills'][0]
            total_red_assists += player_df['assists'][0]
            total_red_gold_earned += player_df['gold_earned'][0]
            total_red_minions_killed += player_df['total_minions_killed'][0]
            total_red_turret_kills += player_df['turret_kills'][0]
            total_red_dragon_kills += player_df['dragon_kills'][0]
            total_red_damage_dealt_to_champions += player_df['total_damage_dealt_champions'][0]
            total_red_baron_kills += player_df['baron_kills'][0]
            total_red_champ_level += player_df['champ_level'][0]
            total_red_inhibitor_kills += player_df['inhibitor_kills'][0]
            total_red_items_purchased += player_df['items_purchased'][0]

        
    match_series = pd.Series({
        'match_id': match_id,
        'game_mode': game_mode,
        'queue_id': queue_id,
        'platform_id': platform_id,
        'game_duration_in_ms': game_duration_in_ms,
        'game_duration_in_min': game_duration_in_min,


        'win': win,
        'surrender': surrender,

        'total_blue_kills': total_blue_kills,
        'total_red_kills': total_red_kills,
        'total_blue_assists': total_blue_assists,
        'total_red_assists': total_red_assists,
        'total_blue_gold_earned': total_blue_gold_earned,
        'total_red_gold_earned': total_red_gold_earned,
        'total_blue_minions_killed': total_blue_minions_killed,
        'total_red_minions_killed': total_red_minions_killed,
        'total_blue_turret_kills': total_blue_turret_kills,
        'total_red_turret_kills': total_red_turret_kills,
        'total_blue_dragon_kills': total_blue_dragon_kills,
        'total_red_dragon_kills': total_red_dragon_kills,
        'total_blue_damage_dealt_to_champions': total_blue_damage_dealt_to_champions,
        'total_red_damage_dealt_to_champions': total_red_damage_dealt_to_champions,
        'total_blue_baron_kills': total_blue_baron_kills,
        'total_red_baron_kills': total_red_baron_kills,
        'avg_blue_champ_level': (round(total_blue_champ_level / 5, 2)),
        'avg_red_champ_level': (round(total_red_champ_level / 5, 2)),
        'total_blue_inhibitor_kills': total_blue_inhibitor_kills,
        'total_red_inhibitor_kills': total_red_inhibitor_kills,
        'total_blue_items_purchased': total_blue_items_purchased,
        'total_red_items_purchased': total_red_items_purchased
    })

    return match_series

In [ ]:
# sciagniecie nazw graczy z rangi diament ze strony op.gg
def get_nicknames_from_diamond(page_num):
    opgg = requests.get(f'https://www.op.gg/leaderboards/tier?tier=diamond&page={page_num}')
    soup = BeautifulSoup(opgg.text, 'html.parser')
    nicknames = soup.findAll('a', attrs={'class':'summoner-link'})
    return [tuple(nickname.text.split('#')) for nickname in nicknames] 

In [ ]:
nickname_tag_tuples = get_nicknames_from_diamond(4)
# nickname_tag_tuples = nickname_tag_tuples[:25]

In [ ]:
players_puuids = []
for nickname in nickname_tag_tuples:
    try:
        time.sleep(1.5)
        puuid = get_player_puuid(game_name=nickname[0], tag_line=nickname[1])
        players_puuids.append(puuid)
    except:
        pass

players_puuids

In [ ]:
print(len(players_puuids))
matches = set()
for puuid in players_puuids:
    time.sleep(1.5)
    try:
        matches_history = get_match_history_ids(puuid=puuid)
        print(matches_history)
        for match in matches_history:
            matches.add(match)
    except:
        pass

In [ ]:
all_games_len = len(matches)
print(all_games_len)
ranked_games = set()

for index, match in enumerate(matches, start=1):
    time.sleep(1.5)
    print(f'\r{index}/{all_games_len}', end=' - ')
    try:
        if check_if_ranked(match):
            ranked_games.add(match)
            print('ranked added')
        else:
            print()
    except Exception as e:
        print(e)

In [ ]:
ranked_games_len = len(ranked_games)
print(ranked_games_len)
ranked_games_df = pd.DataFrame()

for index, match in enumerate(ranked_games):
    time.sleep(1.5)
    try:
        ranked_games_df[str(index)] = process_match_json(match)
        print(f'{index+1}/{ranked_games_len}')
    except Exception as e:
        print(e)

In [ ]:
file_name = 'lol_ranked_games_0.csv'
if os.path.exists(file_name):
    os.remove(file_name)

ranked_games_df = ranked_games_df.transpose()
ranked_games_df.to_csv(file_name)
print(ranked_games_df.shape)